In [1]:
import dlib
import cv2
import numpy as np

In [2]:
# Hàm chuyển đổi từ định dạng dlib landmarks sang numpy 
def landmarks_to_np(landmarks, dtype="int"):
    # Nhận số lượng mốc của khuôn mặt
    num = landmarks.num_parts
    # Khởi tạo danh sách tọa độ của các điểm x,y
    coords = np.zeros((num, 2), dtype=dtype)
    
    # Tạo vòng lặp 68 điểm trên khuôn mặt
    # chuyển đổi thành 2 tọa độ x,y
    for i in range(0, num):
        coords[i] = (landmarks.part(i).x, landmarks.part(i).y)
    # return danh sách tọa độ (x,y)
    return coords

In [3]:
# Tìm tọa độ đồng tử mắt
def get_centers(img, landmarks):
    
    EYE_LEFT_OUTTER = landmarks[2]
    EYE_LEFT_INNER = landmarks[3]
    EYE_RIGHT_OUTTER = landmarks[0]
    EYE_RIGHT_INNER = landmarks[1]

    x = ((landmarks[0:4]).T)[0] #Lấy các tọa độ x
    y = ((landmarks[0:4]).T)[1] #Lấy các tọa độ y
    A = np.vstack([x, np.ones(len(x))]).T #Khởi tạo mảng A chứa các tọa độ x và đánh dấu 1
    k, b = np.linalg.lstsq(A, y, rcond=None)[0] #Hệ số đường tuyến tính
    
    x_left = (EYE_LEFT_OUTTER[0]+EYE_LEFT_INNER[0])/2 #Trung điểm mắt trái tính từ trong ra ngoài
    x_right = (EYE_RIGHT_OUTTER[0]+EYE_RIGHT_INNER[0])/2 #Trung điểm mắt phải tính từ trong ra ngoài
    LEFT_EYE_CENTER =  np.array([np.int32(x_left), np.int32(x_left*k+b)]) #tọa độ tròng mắt trái
    RIGHT_EYE_CENTER =  np.array([np.int32(x_right), np.int32(x_right*k+b)]) #tọa độ tròng mắt phải
    
    pts = np.vstack((LEFT_EYE_CENTER,RIGHT_EYE_CENTER))
    cv2.polylines(img, [pts], False, (255,0,0), 1) # vẽ đường thẳng nối 2 tròng mắt
    cv2.circle(img, (LEFT_EYE_CENTER[0],LEFT_EYE_CENTER[1]), 3, (0, 0, 255), -1)
    cv2.circle(img, (RIGHT_EYE_CENTER[0],RIGHT_EYE_CENTER[1]), 3, (0, 0, 255), -1)
    
    return LEFT_EYE_CENTER, RIGHT_EYE_CENTER

In [4]:
# Căn chỉnh khuôn mặt với tròng mắt trái và phải
def get_aligned_face(img, left, right):
    desired_w = 256
    desired_h = 256
    desired_dist = desired_w * 0.5

    eyescenter = ((left[0]+right[0])*0.5 , (left[1]+right[1])*0.5)# tọa độ chính giữa 2 mắt
    cv2.circle(img, (int(eyescenter[0]), int(eyescenter[1])), 3, (0, 0, 255), -1) # vẽ vòng tròn tâm giữa mắt
    dx = right[0] - left[0] #độ dài giữa 2 mắt theo trục x
    dy = right[1] - left[1] #độ dài giữa 2 mắt theo trục y
    dist = np.sqrt(dx*dx + dy*dy)#khoảng cách giữa 2 mắt
    scale = desired_dist / dist #tỉ lệ co giãn
    angle = np.degrees(np.arctan2(dy,dx)) # tính góc giữa dx, dy
    M = cv2.getRotationMatrix2D(eyescenter,angle,scale)# tính toán ma trận xoay
    # print(M)
    # cập nhật thành phần ma trận
    tX = desired_w * 0.5
    tY = desired_h * 0.5
    M[0, 2] += (tX - eyescenter[0])
    M[1, 2] += (tY - eyescenter[1])

    aligned_face = cv2.warpAffine(img,M,(desired_w,desired_h))
    
    return aligned_face

In [ ]:
def judge_eyeglass(img):
    img = cv2.GaussianBlur(img, (11,11), 0) #高斯模糊

    sobel_y = cv2.Sobel(img, cv2.CV_64F, 0 ,1 , ksize=-1) #y方向sobel边缘检测
    sobel_y = cv2.convertScaleAbs(sobel_y) #转换回uint8类型
    cv2.imshow('sobel_y',sobel_y)

    edgeness = sobel_y #边缘强度矩阵
    
    #Otsu二值化
    retVal,thresh = cv2.threshold(edgeness,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    #计算特征长度
    d = len(thresh) * 0.5
    x = np.int32(d * 6/7)
    y = np.int32(d * 3/4)
    w = np.int32(d * 2/7)
    h = np.int32(d * 2/4)

    x_2_1 = np.int32(d * 1/4)
    x_2_2 = np.int32(d * 5/4)
    w_2 = np.int32(d * 1/2)
    y_2 = np.int32(d * 8/7)
    h_2 = np.int32(d * 1/2)
    
    roi_1 = thresh[y:y+h, x:x+w] #提取ROI
    roi_2_1 = thresh[y_2:y_2+h_2, x_2_1:x_2_1+w_2]
    roi_2_2 = thresh[y_2:y_2+h_2, x_2_2:x_2_2+w_2]
    roi_2 = np.hstack([roi_2_1,roi_2_2])
    
    measure_1 = sum(sum(roi_1/255)) / (np.shape(roi_1)[0] * np.shape(roi_1)[1])#计算评价值
    measure_2 = sum(sum(roi_2/255)) / (np.shape(roi_2)[0] * np.shape(roi_2)[1])#计算评价值
    measure = measure_1*0.3 + measure_2*0.7
    
    cv2.imshow('roi_1',roi_1)
    cv2.imshow('roi_2',roi_2)
    print(measure)
    
    #根据评价值和阈值的关系确定判别值
    if measure > 0.15:#阈值可调，经测试在0.15左右
        judge = True
    else:
        judge = False
    print(judge)
    return judge


In [5]:
predictor_path = "./data/shape_predictor_5_face_landmarks.dat" # Đường dẫn file dữ liệu mô hình đã được phân tích
detector = dlib.get_frontal_face_detector() # Dò khuôn mặt
predictor = dlib.shape_predictor(predictor_path) # Dự đoán các điểm trên khuôn mặt
predictor

In [6]:
cap = cv2.VideoCapture(0) #Khởi tạo camera usb
while(cap.isOpened()):
    # Đọc video
    _, img = cap.read()

    #Chuyển sang thanh màu xám
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Phát hiện khuôn mặt
    rects = detector(gray, 1)

    for i, rect in enumerate(rects):
        # Lấy tọa độ khuôn mặt
        x_face = rect.left()
        y_face = rect.top()
        w_face = rect.right() - x_face
        h_face = rect.bottom() - y_face
        
        # Vẽ viền cho khuôn mặt và thêm chú thích
        cv2.rectangle(img, (x_face,y_face), (x_face+w_face,y_face+h_face), (0,255,0), 2)
        cv2.putText(img, "Face #{}".format(i + 1), (x_face - 10, y_face - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

        # Đánh dấu các điểm mốc trên khuôn mặt     
        landmarks = predictor(gray, rect)
        landmarks = landmarks_to_np(landmarks)
        
        for (x, y) in landmarks:
            cv2.circle(img, (x, y), 2, (0, 0, 255), -1)
        
        # Lấy các vị trí trung tâm của mắt
        LEFT_EYE_CENTER, RIGHT_EYE_CENTER = get_centers(img, landmarks)
        # Căn chỉnh khuôn mặt
        aligned_face = get_aligned_face(gray, LEFT_EYE_CENTER, RIGHT_EYE_CENTER)
        cv2.imshow("aligned_face #{}".format(i + 1), aligned_face)

        # phân tích đeo kính hoặc k đeo kính
        judge = judge_eyeglass(aligned_face)
        if judge == True:
            cv2.putText(img, "With Glasses", (x_face + 100, y_face - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
        else:
            cv2.putText(img, "No Glasses", (x_face + 100, y_face - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
    
    cv2.imshow("Result", img)
    k = cv2.waitKey(5) & 0xFF
    if k==27:   # Nhấn nút ESC thì thoát
        break

cap.release()
cv2.destroyAllWindows()